<a href="https://colab.research.google.com/github/priyadarshinikp1/Genomics/blob/main/genomics_cnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copy Number Variant (CNV) calling using GATK

1. PreprocessIntervals
  

🔍 What it does:

Splits the genome into non-overlapping bins (e.g., 1000 bp windows).

🧠 Why:

CNVs are detected by comparing read depth across the genome. Binning makes the genome easier to scan and allows for uniform analysis of coverage differences.

In [ ]:
gatk PreprocessIntervals \
  -R reference.fasta \
  --bin-length 1000 \
  --interval-merging-rule OVERLAPPING_ONLY \
  -O intervals.interval_list

2. AnnotateIntervals
🔍 What it does:

Adds GC content and other metadata to each bin.

🧠 Why:

Read depth is affected by GC bias — regions with very high or low GC content tend to get sequenced unevenly. GC annotation helps GATK normalize for this during denoising.

In [ ]:
gatk AnnotateIntervals \
  -R reference.fasta \
  -L intervals.interval_list \
  -O intervals_annotated.tsv

  --interval-merging-rule OVERLAPPING_ONLY \

3. CollectReadCounts

🔍 What it does:

Counts the number of reads mapped to each genomic bin.

🧠 Why:

CNVs cause changes in read depth. For example:

Deletions → fewer reads in a region

Duplications → more reads

This is the core input data for CNV calling.

In [ ]:
gatk CollectReadCounts \
  -I sample.bam \
  -L intervals.interval_list \
  --interval-merging-rule OVERLAPPING_ONLY \
  -O sample.counts.hdf5

4. DenoiseReadCounts

🔍 What it does:

Normalizes your read counts to remove noise: GC bias, Global depth variability, Sequencing artifacts

🧠 Why:

Raw read counts are noisy. Denoising helps reveal the true copy number signal, making deletions and duplications easier to detect accurately — especially without a PoN.

If you have a Panel of Normals (--count-panel-of-normals), it uses those normals to better subtract systematic biases. But even without one, GATK applies GC and depth correction.

In [ ]:
gatk DenoiseReadCounts \
  -I sample.counts.hdf5 \
  --standardized-copy-ratios sample.standardizedCR.tsv \
  --denoised-copy-ratios sample.denoisedCR.tsv


5. ModelSegments

🔍 What it does:

Segments the genome into contiguous regions of similar copy number.

🧠 Why:

Rather than calling CNVs in every small bin, it groups bins with similar copy ratios into meaningful regions (segments). This step creates the actual CNV profiles of the sample.

In [ ]:
gatk ModelSegments \
  --denoised-copy-ratios sample.denoisedCR.tsv \
  --output output_dir \
  --output-prefix sample

6. CallCopyRatioSegments

🔍 What it does:

Applies statistical rules to label segments as: Deletion, Duplication, Neutral

🧠 Why:

This step finalizes the CNV calls by interpreting the copy ratio values and turning them into biologically meaningful variants.

Output is in .seg format, which you can convert to .bed or annotate with gene info.


In [ ]:
gatk CallCopyRatioSegments \
  -I output_dir/sample.modelFinal.seg \
  -O sample.called.seg



🧬 CNV Annotation with Gene Names

To annotate your CNVs (e.g., from sample.called.seg) with gene information, here’s how you can do it:



🧰 What You Need

Your CNV calls in BED format

 → Convert from .seg if needed

A gene annotation file (e.g., GENCODE or RefSeq) in BED or GTF format

 You can download one from: GENCODE BED, UCSC Table Browser (RefSeq or Ensembl)



📦 Step-by-Step: Annotate CNVs with bedtools

1. Convert .seg to .bed  

In [ ]:
awk 'NR>1 {print $2"\t"$3"\t"$4"\t"$5}' sample.called.seg > sample.cnv.bed

2. Intersect CNVs with Genes

This gives you: CNV coordinates, Gene names overlapped, Can also add gene functions manually if needed

In [ ]:
bedtools intersect -a sample.cnv.bed -b genes.bed -wa -wb > sample.cnv.annotated.bed

VISUALIZATION OF CNVs

✅ 1. Plot CNVs (Visualize Copy Ratios)

Use PlotDenoisedCopyRatios to generate plots for your CNV calls:

In [ ]:
java -jar ./gatk/build/libs/gatk.jar PlotDenoisedCopyRatios \
  --standardized-copy-ratios sample.standardizedCR.tsv \
  --denoised-copy-ratios sample.denoisedCR.tsv \
  --sequence-dictionary ./ucsref/hg38.dict \
  --output output_dir \
  --output-prefix sample

✅ 2. Convert .seg to .bed for annotation

GATK .seg format → BED format:
  
Explanation:

Skips header (tail -n +2)

Outputs: chr, start, end, copy ratio

In [ ]:
tail -n +2 sample.called.seg | awk 'BEGIN{OFS="\t"} {print $1, int($2), int($3), $4}' > sample.cnv.bed



✅ 3. Download RefSeq Gene BED (UCSC Table Browser)

Go to: https://genome.ucsc.edu/cgi-bin/hgTables

Settings:

clade: Mammal

genome: Human

assembly: hg38

group: Genes and Gene Predictions

track: RefSeq Genes

table: refGene

output format: BED - browser extensible data

output file: refseq_genes.bed

Select:

✅ “Create one BED record per” → Whole gene

✅ Add 200 bases upstream/downstream (optional)

Then click "get output" → download file.

✅ 4. Annotate CNVs with Gene Names using bedtools

Make sure you have bedtools installed:
  

This gives:

Your CNV regions

Overlapping genes (from RefSeq)

In [ ]:
bedtools intersect -a sample.cnv.bed -b refseq_genes.bed -wa -wb > sample.cnv.annotated.bed

✅ Result

You now have:

📁 sample.cnv.annotated.bed with CNVs and gene names

📈 Plots in output_dir/ for each chromosome

🧬 sample.called.seg, .tsv, and .bed for downstream analysis